## 4.3.3 피드포워드 네트워크
* 책의 코드가 동작하도록 원서의 내용에서 일부 코드를 추가했습니다.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    """
    Transformer에서 사용되는 Position-wise Feed-Forward Network
    
    구조:
    - Linear layer (input_dim → hidden_dim)
    - SeLU activation function
    - Linear layer (hidden_dim → input_dim)
    
    Args:
        input_dim (int): 입력 차원 (보통 d_model과 같음)
        hidden_dim (int): 은닉층 차원 (보통 input_dim의 4배)
    """
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        
        # 첫 번째 선형 변환: input_dim → hidden_dim
        self.l1 = nn.Linear(input_dim, hidden_dim)
        
        # 두 번째 선형 변환: hidden_dim → input_dim (원래 차원으로 복원)
        self.l2 = nn.Linear(hidden_dim, input_dim)
        
        # ReLU 활성화 함수 (일반적으로 Transformer에서 많이 사용)
        self.relu = nn.ReLU()
        
        # 또는 GELU를 사용하려면 아래 주석을 해제하세요 (최신 Transformer에서 선호)
        # self.activation = nn.GELU()

    def forward(self, x):
        """
        순전파 과정
        
        Args:
            x (torch.Tensor): 입력 텐서 (batch_size, seq_len, input_dim)
            
        Returns:
            torch.Tensor: 출력 텐서 (batch_size, seq_len, input_dim)
        """
        # 첫 번째 선형 변환 후 ReLU 활성화
        x = self.relu(self.l1(x))
        
        # GELU를 사용하려면 위 줄을 아래로 교체하세요:
        # x = F.gelu(self.l1(x))
        
        # 두 번째 선형 변환 (활성화 함수 없음)
        x = self.l2(x)
        
        return x

# 하이퍼파라미터 설정
input_dim = 512    # 입력/출력 차원 (d_model)
hidden_dim = 2048  # 은닉층 차원 (보통 d_model * 4)
batch_size = 2     # 배치 크기
seq_len = 10       # 시퀀스 길이

# 더미 입력 데이터 생성 (예: Transformer의 attention layer 출력)
inputs = torch.randn(batch_size, seq_len, input_dim)
print(f"입력 텐서 형태: {inputs.shape}")

# FeedForward 네트워크 인스턴스 생성
feed_forward = FeedForward(input_dim, hidden_dim)

# 모델 파라미터 수 계산
total_params = sum(p.numel() for p in feed_forward.parameters())
print(f"총 파라미터 수: {total_params:,}")

# 추론 모드에서 순전파 실행
feed_forward.eval()  # 평가 모드로 설정
with torch.no_grad():  # 추론 모드 (gradient 계산 안함)
    outputs_inference = feed_forward(inputs)

print(f"출력 텐서 형태: {outputs_inference.shape}")
print(f"입력과 출력 형태가 같은지: {inputs.shape == outputs_inference.shape}")

# 네트워크 구조 출력
print(f"\n네트워크 구조:")
print(feed_forward)

# 각 레이어별 가중치 형태 확인
print(f"\n레이어별 가중치 형태:")
print(f"l1.weight: {feed_forward.l1.weight.shape}")
print(f"l1.bias: {feed_forward.l1.bias.shape}")
print(f"l2.weight: {feed_forward.l2.weight.shape}")
print(f"l2.bias: {feed_forward.l2.bias.shape}")

# 간단한 예시: 학습 모드에서의 gradient 계산
print(f"\n학습 예시:")
feed_forward.train()  # 학습 모드로 설정

# gradient 계산을 위해 입력 텐서에 requires_grad=True 설정
inputs_train = torch.randn(batch_size, seq_len, input_dim, requires_grad=True)

# 순전파 (gradient 계산 활성화)
outputs = feed_forward(inputs_train)

# 가상의 target과 loss 계산
target = torch.randn_like(outputs)
criterion = nn.MSELoss()
loss = criterion(outputs, target)

print(f"Loss: {loss.item():.4f}")

# Backward pass
loss.backward()
print("Gradient 계산 완료!")

# 네트워크 파라미터의 gradient 확인
if feed_forward.l1.weight.grad is not None:
    print(f"l1.weight gradient norm: {feed_forward.l1.weight.grad.norm().item():.4f}")
    print(f"l2.weight gradient norm: {feed_forward.l2.weight.grad.norm().item():.4f}")

# 입력 텐서의 gradient도 확인 (requires_grad=True로 설정했기 때문)
if inputs_train.grad is not None:
    print(f"입력 텐서 gradient norm: {inputs_train.grad.norm().item():.4f}")

# Optimizer 사용 예시
print(f"\n옵티마이저 사용 예시:")
optimizer = torch.optim.Adam(feed_forward.parameters(), lr=0.001)

# Gradient 초기화
optimizer.zero_grad()

# 새로운 forward pass
outputs = feed_forward(inputs_train)
loss = criterion(outputs, target)

# Backward pass
loss.backward()

# 파라미터 업데이트
optimizer.step()

print(f"파라미터 업데이트 완료! 새로운 Loss: {loss.item():.4f}")

입력 텐서 형태: torch.Size([2, 10, 512])
총 파라미터 수: 2,099,712
출력 텐서 형태: torch.Size([2, 10, 512])
입력과 출력 형태가 같은지: True

네트워크 구조:
FeedForward(
  (l1): Linear(in_features=512, out_features=2048, bias=True)
  (l2): Linear(in_features=2048, out_features=512, bias=True)
  (relu): ReLU()
)

레이어별 가중치 형태:
l1.weight: torch.Size([2048, 512])
l1.bias: torch.Size([2048])
l2.weight: torch.Size([512, 2048])
l2.bias: torch.Size([512])

학습 예시:
Loss: 1.0602
Gradient 계산 완료!
l1.weight gradient norm: 0.1919
l2.weight gradient norm: 0.4036
입력 텐서 gradient norm: 0.0051

옵티마이저 사용 예시:
파라미터 업데이트 완료! 새로운 Loss: 1.0602
